In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(0)

In [11]:
# definição dos grupos de filiais pra facilitar a vida

df = pd.read_csv("dados_simulacao/distancias_todas_combinacoes.csv")

CD = 'CENTRO DE DISTRIBUICAO'

todas_com_cd = df

todas_sem_cd = df[(df['Filial_A'] != CD) 
                      & (df['Filial_B'] != CD)]

rj_com_cd = df[(df['Estado_A'] == 'RJ') &(df['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ

rj_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'RJ')&(todas_sem_cd['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ \{filial = CD}

sp_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'SP')&(todas_sem_cd['Estado_B'] == 'SP')] # uniao de E_A e E_B == SP

cd_sp = df[((df['Estado_A'] == 'SP') & (df['Filial_B'] == CD))
           | ((df['Estado_B'] == 'SP') & (df['Filial_A'] == CD))]
sp_com_cd = pd.concat((cd_sp, sp_sem_cd), ignore_index=True)  # uniao de E_A e E_B == SP  +  {A = CD e E_B = SP}

lojas_A = todas_sem_cd[['Estado_A', 'Filial_A']].rename(columns={'Estado_A': 'Estado', 'Filial_A': 'Filial'})
lojas_B = todas_sem_cd[['Estado_B', 'Filial_B']].rename(columns={'Estado_B': 'Estado', 'Filial_B': 'Filial'})

um_de_cada = pd.concat([lojas_A, lojas_B]) \
                     .drop_duplicates() \
                     .sort_values(by=['Estado', 'Filial']) \
                     .drop_duplicates(subset=['Estado']) \
                     .reset_index(drop=True)

estado_sem_cd = todas_sem_cd[(todas_sem_cd['Filial_A'].isin(um_de_cada['Filial'].tolist())) &
                             (todas_sem_cd['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

cd_filiais = df[(df['Filial_A'] == CD) & (df['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

estado_com_cd = pd.concat([estado_sem_cd, cd_filiais], ignore_index=True)

___

#### Temos uma função $F_A(B)$ = valor de B no ranking de A:

##### 1) Caso A→B (A como provedor):

$$F_A(B) = (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

##### 2) Caso CD→B (CD como provedor): 

$$F_{CD}(B) = α * (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

onde $α$ é um fator preferencia do CD ser o provedor.

___

#### Utilize uma das variáveis para a simulação:

* `todas_sem_cd` para utilizar todas as filiais do Brasil.
* `todas_com_cd` para utilizar todas as filiais do Brasil e o Centro de Distribuição.
* `rj_sem_cd` para utilizar todas as filiais do Rio de Janeiro.
* `rj_com_cd` para utilizar todas as filiais do Rio de Janeiro e o Centro de Distribuição.
* `sp_sem_cd` para utilizar todas as filiais de São Paulo.
* `sp_com_cd` para utilizar todas as filiais de São Paulo e o Centro de Distribuição.
* `estado_sem_cd` para utilizar uma filial e cada estado.
* `estado_com_cd` para utilizar uma filial e cada estado e o Centro de Distribuição.

#### Não esqueça de especificar se o CD está incluso ou não (True/False)

In [ ]:
# Defina todas as variáveis abaixo!

filiais = estado_sem_cd # qual conjunto de variáveis usar 
cd = False # booleano se considera ou não o CD no grafo # acho que nem vai ser necessário

c = 1 # constante de tempo de transporte por km 
k_filial = 0.5  # constante de custo do frete em reais por km saindo de A
k_cd = 0.2  # constante de custo do frete em reais por km saindo do CD
p = 2  # peso se a filial está em ruptura
q = 1  # peso se a filial está em falta
alpha = 2  # fator de preferencia do CD ser o provedor

params = {'c': c, 'k_filial': k_filial, 'k_cd': k_cd, 'p': p, 'q': q, 'alpha': alpha}


In [4]:
def get_distancia(a, b, df_distancias):
    if a == b:
        return 0
    dist_df = df_distancias[
        ((df_distancias['Filial_A'] == a) & (df_distancias['Filial_B'] == b)) |
        ((df_distancias['Filial_A'] == b) & (df_distancias['Filial_B'] == a))
    ]
    if dist_df.empty:
        return np.inf
    return dist_df['Distancia_km'].values[0]

In [ ]:
def classificar_filiais(df_master):
    """
    Classifica um DataFrame mestre (com a nomenclatura Est_disp/Alvo)
    em DataFrames de provedores e receptoras.

    Argumentos:
    df_master -- DataFrame com as colunas: 
                 ['Filial', 'Tipo', 'Est_disp', 'Alvo']
    
    Retorna:
    (df_provedores, df_receptoras) -- Uma tupla contendo os dois DataFrames.
    """
    
    # Validação das colunas necessárias
    colunas_necessarias = ['Filial', 'Tipo', 'Est_disp', 'Alvo']
    if not all(col in df_master.columns for col in colunas_necessarias):
        raise ValueError(f"O DataFrame mestre precisa ter as colunas: {colunas_necessarias}")

    # --- 1. Criação do DataFrame de PROVEDORES ---
    df_prov = df_master.copy()
    
    cond_cd = (df_prov['Tipo'] == 'CD')
    excesso_filial = (df_prov['Est_disp'] - df_prov['Alvo']).clip(lower=0)
    excesso_cd = df_prov['Est_disp']

    df_prov['Volume_em_excesso'] = np.where(
        cond_cd,
        excesso_cd,      # Regra do CD
        excesso_filial   # Regra da Filial
    )
    
    df_provedores = df_prov[df_prov['Volume_em_excesso'] > 0].copy()
    df_provedores = df_provedores[['Filial', 'Tipo', 'Volume_em_excesso']]

    
    # --- 2. Criação do DataFrame de RECEPTORAS ---
    df_rec = df_master[df_master['Tipo'] == 'Filial'].copy()

    cond_receptora = df_rec['Est_disp'] < df_rec['Alvo']
    df_receptoras = df_rec[cond_receptora].copy()
    
    # --- Cálculos para as Receptoras ---
    df_receptoras['Volume_em_falta'] = (
        df_receptoras['Alvo'] - df_receptoras['Est_disp']
    )
    
    df_receptoras['ContRup'] = np.where(
        df_receptoras['Est_disp'] == 0, 1, 0
    )
    
    df_receptoras['ContFalta'] = np.where(
        df_receptoras['Est_disp'] > 0, 1, 0
    )
    
    colunas_finais_rec = ['Filial', 'Volume_em_falta', 'ContRup', 'ContFalta']
    df_receptoras = df_receptoras[colunas_finais_rec]
    
    return df_provedores, df_receptoras

In [6]:
def calcular_ranking_F(
    tipo_provedor, 
    distancia, 
    cont_rup, 
    cont_falta, 
    params, 
):
    """
    Calcula o score F_A(B) ou F_CD(B) para um par (A, B),
    usando os dados pré-processados.
    
    'params' é o dicionário com seus parâmetros:
    params = {
        'c': 0.5, 'k_filial': 2.0, 'k_cd': 1.0, 
        'p': 10, 'q': 5, 'alpha': 1.5
    }
    """
    
    if distancia == np.inf or distancia == 0:
        return 0 

    # --- 1. Calcular Componentes Base ---
    leadtime = params['c'] * distancia
    
    if tipo_provedor == 'CD':
        custo_frete = params['k_cd'] * distancia
    else:
        custo_frete = params['k_filial'] * distancia
        
    if leadtime == 0 or custo_frete == 0:
        return 0

    # --- 2. Calcular Componente de Urgência ---
    # Os booleanos agora vêm prontos (0 ou 1)
    urgencia = (cont_rup * params['p']) + (cont_falta * params['q'])
    
    # Se urgência for 0 (nem ruptura, nem falta), o score é 0
    if urgencia == 0:
        return 0
    
    # --- 3. Calcular Fórmula Final ---
    fator_eficiencia = (1 / leadtime) + (1 / custo_frete)
    score_base = fator_eficiencia * urgencia
    
    if tipo_provedor == 'CD':
        score_final = params['alpha'] * score_base
    else:
        score_final = score_base
        
    return score_final

In [7]:
def montar_matriz_T(df_provedores, df_receptoras, df_distancias, params):
    """
    Monta a Matriz T (DataFrame) onde T[i, j] = F_Ai(B_j).
    
    - df_provedores: DataFrame de classificar_filiais_adaptada
    - df_receptoras: DataFrame de classificar_filiais_adaptada
    """
    
    matriz_T = pd.DataFrame(
        index=df_provedores['Filial'], 
        columns=df_receptoras['Filial'],
        dtype=float
    )

    # Itera sobre cada PROVEDOR (linhas da matriz)
    for idx_a, provedor in df_provedores.iterrows():
        filial_a = provedor['Filial']
        tipo_a = provedor['Tipo']

        # Itera sobre cada RECEPTORA (colunas da matriz)
        for idx_b, receptora in df_receptoras.iterrows():
            filial_b = receptora['Filial']
            
            # Pega os dados de urgência da receptora
            cont_rup_b = receptora['ContRup']
            cont_falta_b = receptora['ContFalta']
            
            # Busca a distância uma única vez
            dist = get_distancia(filial_a, filial_b, df_distancias)
            
            # Calcula o score do par (A, B)
            score = calcular_ranking_F(
                tipo_provedor=tipo_a,
                distancia=dist,
                cont_rup=cont_rup_b,
                cont_falta=cont_falta_b,
                params=params
            )
            
            # Preenche a matriz
            matriz_T.loc[filial_a, filial_b] = score
            
    return matriz_T

In [8]:
def criar_listas_distribuicao(matriz_T):
    """
    Recebe a matriz de scores T[A, B] e cria as listas de alocação 
    para cada provedor (A), contendo as receptoras (B) que o 
    escolheram como melhor opção, ordenadas por score.
    """
    
    # 1. Encontrar o melhor Provedor (A) para cada Receptora (B)
    #    "pegamos o maior valor de cada coluna"
    #    Isso cria uma Série: Index=Receptora (B), Valor=Provedor (A)
    melhores_provedores_para_B = matriz_T.idxmax(axis=0)
    
    # 2. Inverter o grafo para criar as listas por Provedor (A)
    #    (Seu passo: "Criamos uma lista para cada A_i")
    atribuicoes = {}
    
    # Itera sobre a série (B_j, A_i)
    for receptora, provedor in melhores_provedores_para_B.items():
        
        # Pega o score que B deu para A
        score = matriz_T.loc[provedor, receptora]
        
        # Se o score for 0 ou NaN (ex: rota impossível), B não escolhe ninguém
        if pd.isna(score) or score == 0:
            continue
            
        # Adiciona B na lista de A
        if provedor not in atribuicoes:
            atribuicoes[provedor] = []
            
        atribuicoes[provedor].append({
            'receptora': receptora,
            'score': score
        })

    # 3. Ordenar as listas de B para cada A (maior score primeiro)
    for provedor in atribuicoes:
        atribuicoes[provedor].sort(key=lambda x: x['score'], reverse=True)

    return atribuicoes

In [9]:
def executar_alocacao(
    listas_distribuicao, 
    df_provedores, 
    df_receptoras
):
    """
    Executa a alocação de estoque com base nas listas de prioridade.
    
    Retorna:
    - transacoes: (list) Um log de todos os envios a serem feitos.
    - df_provedores_final: (DataFrame) O estoque restante dos provedores.
    - df_receptoras_final: (DataFrame) A demanda restante das receptoras.
    """
    
    # 1. Preparar o estado (estoques e demandas) para podermos alterá-los
    # Usamos .copy() para não modificar os DataFrames originais
    provedores_final = df_provedores.set_index('Filial').copy()
    receptoras_final = df_receptoras.set_index('Filial').copy()
    
    # Lista para registrar o que foi feito
    transacoes = []

    # 2. Iterar sobre cada Provedor (A) e sua lista de Receptoras (B)
    for provedor, lista_prioridade in listas_distribuicao.items():
        
        # Pega o estoque que este provedor (A) tem para enviar
        estoque_disponivel_A = provedores_final.loc[provedor, 'Volume_em_excesso']

        # 3. Iterar sobre as Receptoras (B) na ordem de prioridade
        for item in lista_prioridade:
            receptora = item['receptora']
            
            # (Parada 1) Se o estoque de A acabou, ele para de distribuir
            if estoque_disponivel_A <= 0:
                break 
            
            # Pega a demanda (o quanto B ainda precisa)
            demanda_B = receptoras_final.loc[receptora, 'Volume_em_falta']
            
            # Se B não precisa de mais nada, pula para a próxima
            if demanda_B <= 0:
                continue

            # 4. Calcular a quantidade a ser enviada
            # É o valor mínimo entre o que A tem e o que B precisa
            quantidade_a_enviar = min(estoque_disponivel_A, demanda_B)
            
            # 5. Registrar a Transação
            transacoes.append({
                'De_Provedor': provedor,
                'Para_Receptora': receptora,
                'Quantidade': quantidade_a_enviar,
                'Score_Prioridade': item['score']
            })
            
            # 6. Atualizar os valores (Seus passos 7.1 e 7.2)
            
            # Reduz o estoque de A
            estoque_disponivel_A -= quantidade_a_enviar
            
            # Reduz a demanda de B
            demanda_B -= quantidade_a_enviar
            
            # Salva os novos valores nos DataFrames
            provedores_final.loc[provedor, 'Volume_em_excesso'] = estoque_disponivel_A
            receptoras_final.loc[receptora, 'Volume_em_falta'] = demanda_B
            
        # (Parada 2) O loop de B's terminou, passa para o próximo A
        
    # Reseta o índice para retornar DataFrames limpos
    return transacoes, provedores_final.reset_index(), receptoras_final.reset_index()

In [10]:
# --- 1. Dados Iniciais ---
dados_mestre = pd.read_csv('filiais_estoque.csv')
df_master = pd.DataFrame(dados_mestre)

df_distancias_simulado = estado_sem_cd

params = {
    'c': 0.5, 'k_filial': 2.0, 'k_cd': 1.0, 
    'p': 10, 'q': 5, 'alpha': 1.5
}
# --- 2. Passo 1: Classificar ---
provedores, receptoras = classificar_filiais(df_master)

print("--- Provedores (Início) ---")
print(provedores)
# Ex: CD-SP (500), Filial-RJ (15)

print("\n--- Receptoras (Início) ---")
print(receptoras)
# Ex: Filial-MG (falta 22, Falta), Filial-BA (falta 10, Ruptura), Filial-AM (falta 10, Falta)


# --- 3. Passo 2: Montar a Matriz T ---
matriz_T = montar_matriz_T(
    provedores, 
    receptoras, 
    df_distancias_simulado, 
    params
)
print("\n--- Matriz T (Scores) ---")
print(matriz_T.round(4))


# --- 4. Passo 3: Criar Listas de Distribuição ---
listas_dist = criar_listas_distribuicao(matriz_T)

print("\n--- Listas de Distribuição (A vai enviar para B) ---")
import json
print(json.dumps(listas_dist, indent=2))
# Exemplo de Saída:
# {
#   "CD-SP": [
#     { "receptora": "Filial-MG", "score": 0.0245 },
#     { "receptora": "Filial-BA", "score": 0.0091 }
#   ],
#   "Filial-RJ": [
#     { "receptora": "Filial-AM", "score": 0.0019 }
#   ]
# }


# --- 5. Passo 4: Executar a Alocação ---
transacoes, provedores_final, receptoras_final = executar_alocacao(
    listas_dist, 
    provedores, 
    receptoras
)

print("\n--- PLANO DE AÇÃO (Transações) ---")
print(pd.DataFrame(transacoes))
# Exemplo de Saída:
#     De_Provedor Para_Receptora  Quantidade  Score_Prioridade
# 0       CD-SP      Filial-MG          22            0.0245
# 1       CD-SP      Filial-BA          10            0.0091
# 2   Filial-RJ      Filial-AM          10            0.0019

print("\n--- Provedores (Final) ---")
print(provedores_final)
# Ex: CD-SP (ficou com 468), Filial-RJ (ficou com 5)

print("\n--- Receptoras (Final) ---")
print(receptoras_final)
# Ex: Todas com Volume_em_falta = 0

--- Provedores (Início) ---
      Filial    Tipo  Volume_em_excesso
0  Filial_PA  Filial                  3
3  Filial_RR  Filial                 12
4  Filial_GO  Filial                  8
5  Filial_SP  Filial                  6
8  Filial_AP  Filial                  1
9  Filial_DF  Filial                 13

--- Receptoras (Início) ---
      Filial  Volume_em_falta  ContRup  ContFalta
1  Filial_TO                4        0          1
2  Filial_MT                9        1          0
6  Filial_ES                1        0          1
7  Filial_PB                2        0          1

--- Matriz T (Scores) ---
Filial     Filial_TO  Filial_MT  Filial_ES  Filial_PB
Filial                                               
Filial_PA        0.0        0.0        0.0        0.0
Filial_RR        0.0        0.0        0.0        0.0
Filial_GO        0.0        0.0        0.0        0.0
Filial_SP        0.0        0.0        0.0        0.0
Filial_AP        0.0        0.0        0.0        0.0
Filial_D